In [1]:

"""
Batch runner to generate combined PR curve figures.

- Intra-model: ARIMA variants together; XGBoost variants together.
- Inter-model: hand-picked best ARIMA vs XGBoost per satellite/feature.

Relies on PR curve CSVs saved under:
  ../arima_logs/prcurves/...
  ../xgb_logs/prcurves/...
and writes figures to:
  ../images/combined_pr/{intra|inter}/<Satellite>/<Feature>_<model>.png
"""

import sys
sys.path.append("../utils")
import matplotlib.pyplot as plt
from combined_pr_plotter import plot_combined_pr_curves

# Global styling

plt.rcParams.update({
    'font.family': 'Times New Roman',
    'font.size': 21,
    'axes.titlesize': 21,
    'axes.labelsize': 19,
    'xtick.labelsize': 19,
    'ytick.labelsize': 19,
    'legend.fontsize': 20
})

# Inputs
satellites = ["Sentinel-3A", "Fengyun-2F", "Fengyun-4A", "SARAL", "Haiyang-2A"]
features = ["Brouwer mean motion", "eccentricity", "mean anomaly"]
match_days_list = [3, 5]

# Labels (intra-model)
arima_modes = ["raw", "cleaned", "smoothed"]
arima_labels = {
    "raw": "Raw",
    "cleaned": "Cleaned",
    "smoothed": "Smoothed (win=7)"
}

xgb_modes = ["xgb_raw", "xgb_cleaned", "multivar_raw", "multivar_cleaned"]

xgb_labels = {
    "xgb_raw": "Univar Raw",
    "xgb_cleaned": "Univar Cleaned",
    "multivar_raw": "Multivar Raw",
    "multivar_cleaned": "Multivar Cleaned"
}

def resolve_log_dir(mode):
    """
    Return the base logs directory for a given mode.
    XGB (including multivariate) -> ../xgb_logs
    ARIMA (raw/cleaned/smoothed) -> ../arima_logs
    """
    if mode.startswith("xgb_") or "multivar" in mode:
        return "../xgb_logs"
    return "../arima_logs"


# Intra-model : ARIMA
print("\n=== Intra-model ARIMA PR Plots ===")
for sat in satellites:
    for feat in features:
        log_dirs = {mode: resolve_log_dir(mode) for mode in arima_modes}
        try:
            plot_combined_pr_curves(
                satellite=sat,
                feature=feat,
                modes=arima_modes,
                match_days_list=match_days_list,
                log_dirs=log_dirs,
                save_root="../images/combined_pr/intra",
                mode_labels=arima_labels,
                model_type="ARIMA"
            )
        except Exception as e:
            print(f"[FAIL] Intra ARIMA – {sat} – {feat}: {e}")

# Intra-model :  XGBoost
print("\n=== Intra-model XGBoost PR Plots ===")
for sat in satellites:
    for feat in features:
        log_dirs = {mode: resolve_log_dir(mode) for mode in xgb_modes}
        
        try:
            plot_combined_pr_curves(
                satellite=sat,
                feature=feat,
                modes=xgb_modes,
                match_days_list=match_days_list,
                log_dirs=log_dirs,
                save_root="../images/combined_pr/intra",
                mode_labels=xgb_labels,
                model_type="XGBoost"
            )
        except Exception as e:
            print(f"[FAIL] Intra XGB – {sat} – {feat}: {e}")



=== Intra-model ARIMA PR Plots ===
[SKIP] Missing PR curve: ../arima_logs/prcurves/Sentinel-3A_Brouwer mean motion_cleaned_mdays3.csv
[SKIP] Missing PR curve: ../arima_logs/prcurves/Sentinel-3A_Brouwer mean motion_cleaned_mdays5.csv
[OK] Saved: ../images/combined_pr/intra/Sentinel-3A/Brouwer_mean_motion_arima.png
[SKIP] Missing PR curve: ../arima_logs/prcurves/Sentinel-3A_eccentricity_cleaned_mdays3.csv
[SKIP] Missing PR curve: ../arima_logs/prcurves/Sentinel-3A_eccentricity_cleaned_mdays5.csv
[OK] Saved: ../images/combined_pr/intra/Sentinel-3A/eccentricity_arima.png
[SKIP] Missing PR curve: ../arima_logs/prcurves/Sentinel-3A_mean anomaly_cleaned_mdays3.csv
[SKIP] Missing PR curve: ../arima_logs/prcurves/Sentinel-3A_mean anomaly_cleaned_mdays5.csv
[OK] Saved: ../images/combined_pr/intra/Sentinel-3A/mean_anomaly_arima.png
[SKIP] Missing PR curve: ../arima_logs/prcurves/Fengyun-2F_Brouwer mean motion_cleaned_mdays3.csv
[SKIP] Missing PR curve: ../arima_logs/prcurves/Fengyun-2F_Brouwer m

In [2]:
#Inter-model: manual best picks

# Map: satellite -> feature -> (ARIMA mode, XGB mode alias)
inter_modes = {
    "Fengyun-2F": {
        "Brouwer mean motion": ("raw", "xgb_raw"),
        "eccentricity": ("cleaned", "xgb_cleaned"),
        "mean anomaly": ("smoothed", "multivar_raw")
    },
    "Fengyun-4A": {
        "Brouwer mean motion": ("raw", "multivar_raw"),
        "eccentricity": ("raw", "xgb_raw"),
        "mean anomaly": ("raw", "xgb_raw")
    },
    "Haiyang-2A": {
        "Brouwer mean motion": ("smoothed", "multivar_raw"),
        "eccentricity": ("raw", "xgb_raw"),
        "mean anomaly": ("smoothed", "multivar_raw")
    },
    "SARAL": {
        "Brouwer mean motion": ("raw", "multivar_raw"),
        "eccentricity": ("smoothed", "xgb_raw"),
        "mean anomaly": ("smoothed", "xgb_raw")
    },
    "Sentinel-3A": {
        "Brouwer mean motion": ("raw", "xgb_raw"),
        "eccentricity": ("smoothed", "multivar_raw"),
        "mean anomaly": ("raw", "multivar_raw")
    }
}

# Legend labels for inter-model comparison
inter_labels = {
    # ARIMA
    "raw": "ARIMA Raw",
    "cleaned": "ARIMA Cleaned",
    "smoothed": "ARIMA Smoothed (win=7)",

    # XGBoost univariate (aliases)
    "xgb_raw": "XGB Univar Raw",
    "xgb_cleaned": "XGB Univar Cleaned",

    # XGBoost multivariate
    "multivar_raw": "XGB Multivar Raw",
    "multivar_cleaned": "XGB Multivar Cleaned"
}

# Aliases -> actual mode strings used in filenames
alias_to_log_mode = {
    "xgb_raw": "raw",
    "xgb_cleaned": "cleaned"
}

def resolve_log_mode(mode):
    """Translate alias (e.g., 'xgb_raw') to the real mode used in CSV filenames."""
    return alias_to_log_mode.get(mode, mode)

# === Run PR curve plots
print("\n=== Inter-model PR Curve Comparison (Manual Picks) ===")
for sat, feature_map in inter_modes.items():
    for feature, (arima_mode, xgb_mode) in feature_map.items():
        modes = [arima_mode, xgb_mode]                        # Aliases
        real_modes = [resolve_log_mode(m) for m in modes]     # For file lookup
        log_dirs = {m: resolve_log_dir(m) for m in modes}     # Keyed by alias
        mode_label_map = {m: inter_labels[m] for m in modes}  # Keyed by alias

        try:
            plot_combined_pr_curves(
                satellite=sat,
                feature=feature,
                modes=modes,  # Pass aliases so legend is correct
                match_days_list=match_days_list,
                log_dirs=log_dirs,
                save_root="../images/combined_pr/inter",
                mode_labels=mode_label_map,
                model_type="Inter-Model"
            )
        except Exception as e:
            print(f"[FAIL] Inter {sat} – {feature}: {e}")



=== Inter-model PR Curve Comparison (Manual Picks) ===
[OK] Saved: ../images/combined_pr/inter/Fengyun-2F/Brouwer_mean_motion_inter-model.png
[OK] Saved: ../images/combined_pr/inter/Fengyun-2F/eccentricity_inter-model.png
[OK] Saved: ../images/combined_pr/inter/Fengyun-2F/mean_anomaly_inter-model.png
[OK] Saved: ../images/combined_pr/inter/Fengyun-4A/Brouwer_mean_motion_inter-model.png
[OK] Saved: ../images/combined_pr/inter/Fengyun-4A/eccentricity_inter-model.png
[OK] Saved: ../images/combined_pr/inter/Fengyun-4A/mean_anomaly_inter-model.png
[OK] Saved: ../images/combined_pr/inter/Haiyang-2A/Brouwer_mean_motion_inter-model.png
[OK] Saved: ../images/combined_pr/inter/Haiyang-2A/eccentricity_inter-model.png
[OK] Saved: ../images/combined_pr/inter/Haiyang-2A/mean_anomaly_inter-model.png
[OK] Saved: ../images/combined_pr/inter/SARAL/Brouwer_mean_motion_inter-model.png
[OK] Saved: ../images/combined_pr/inter/SARAL/eccentricity_inter-model.png
[OK] Saved: ../images/combined_pr/inter/SARAL/